## Familiarization with the data

Imports and the libraries I will be using and data reading:

`“timeStamp”`: is the date time of the mesure.

`“Idflowmeter”`: is the ID of the meter The peculiar series of water measures for two sensor IDs are yet to be explained by Braila.

`“Tot1”`: is the water flow in m3

`“Tot2”`: is the water flow in m3. The difference between Tot1 and Tot2 are yet to be explained by Braila.

`“Analog2”`: is the water pressure of the measure point. The unit is probably Pascal, however not yet confirmed by Braila.

In [1]:
import math
import pandas as pandas
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [2]:
data = pandas.read_json("../data/water_loss_data_set_1.json")

# removing false values
data = data[(data.timeStamp != "0000-00-00 00:00:00" ) & (data.timeStamp != "2000-01-01 00:00:00")]

"""
Missing data in columns: 
tot1: 5086 
tot2: 100635 
analog2: 25

Removing data which is zero, be careful for 'analog2' only 25 values have zero. 
This could be values when some work was done on the pipes so the unhooked them -> pressure 0 ?
""" 
data = data[(data.tot1 != 0 ) & (data.analog2 != 0)]

Processing...
1. find min and max of Tot1, Tot2 and Analog2.

Removed this timestamps from data  
`['0000-00-00 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00']`

In [3]:
data["timeStamp"] =  pandas.to_datetime(data["timeStamp"], format="%Y-%m-%d %H:%M:%S")

# flooring to the nearest hour and grouping by hour
data["timeStamp"] = data["timeStamp"].dt.floor('h')
data_by_hour = data.groupby("timeStamp").mean()
data_by_hour

,tot1,tot2,analog2
timeStamp,,,
2018-11-23 08:00:00,1.410000,0.00,1.110000
2018-11-23 09:00:00,30.987273,0.00,1.103409
2018-11-23 10:00:00,87.809457,0.00,1.099022
2018-11-23 11:00:00,182.008000,0.00,1.101000
2018-11-23 12:00:00,278.252500,0.00,0.960333
...,...,...,...
2020-04-01 10:00:00,977299.898333,-4713.21,1.527333
2020-04-01 11:00:00,977333.727167,-4713.21,1.528500
2020-04-01 12:00:00,977369.184833,-4713.21,1.530500


In [10]:
timestamp = data_by_hour.index.tolist()
timestamp = [str(i) for i in timestamp]

#tot1 = data_by_hour[["tot1"]].to_numpy()
x_axis_data = data_by_hour[["analog2"]].to_numpy()
print("")
print("Max date:", max(timestamp), "\nMin date:", min(timestamp), "\nMedian date:", timestamp[round(len(timestamp)/2)])

#print("\n\nMax date:", timestamp[len(timestamp)-1], "\nMin date:", timestamp[0], "\nMedian date:", 
#      timestamp[round(len(timestamp)/2)])
#print("\n\nFalse values: ", test_array[:10])
print(len(timestamp), len(analog2))


Max date: 2020-04-01 14:00:00 
Min date: 2018-11-23 08:00:00 
Median date: 2019-07-29 12:00:00
11863 11863


time :  
`['0000-00-00 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2018-11-23 07:03:00', '2018-11-23 07:04:00', '2018-11-23 07:06:00', '2018-11-23 07:09:00', '2018-11-23 07:10:00']`
tot1 :  
`[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]`

Max date: `2020-04-01 14:09:00`  
Min date: `0000-00-00 00:00:00` 
Median date: `2019-08-02 02:27:00` 

Max date: `2020-04-01 14:09:00` 
Min date: `0000-00-00 00:00:00` 
Median date: `2019-08-02 02:27:00`


False values:  `[['0000-00-00 00:00:00' -1.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2018-11-23 07:03:00' 0.0]
 ['2018-11-23 07:04:00' 0.0]
 ['2018-11-23 07:06:00' 0.0]
 ['2018-11-23 07:09:00' 0.0]
 ['2018-11-23 07:10:00' 0.0]]`

Checking which data is missing.   
"tot2" attribute contains negative values mostly negative values!! Other do not...

In [11]:
"""
DATA_SIZE = 1390649 
len_tot1 = len(data[(data['tot1'] != 0)])
len_tot2 = len(data[(data['tot2'] != 0)])
len_analog = len(data[(data['analog2'] != 0)])

print("Missing data in columns: ")
print("tot1:", (DATA_SIZE - len_tot1), "\ntot2:", (DATA_SIZE - len_tot2), "\nanalog2:", (DATA_SIZE- len_analog))
""";

In [19]:
# NUM_OF_DATA_TO_PLOT = 10000

fig = plt.figure(figsize=(18, 12), dpi= 100, facecolor='w')
plt.plot(timestamp, x_axis_data)

plt.ylabel('Pressure in bar ?')
plt.xlabel('Date')
plt.xticks([i for i in range(0,len(x_axis_data), 1000)])
plt.rc('xtick',labelsize=8)
"""

plt.xticks([i for count, i in enumerate(timestamp[:NUM_OF_DATA_TO_PLOT]) if count % 150 == 0])
plt.rc('xtick',labelsize=5)
"""

plt.title('Time')
plt.savefig('test.png')
# plt.show()